# 2. 퍼셉트론을 이용한 분류

## 꽃의 구성

* 꽃받침(sepal)
* 꽃잎(petal)

> <img src="https://drive.google.com/uc?id=1dkY2TPvjFcEoNqiieRRFuUF_MTSYPsSx" width="300"/>

## 붓꽃의 종류

> <img src="https://drive.google.com/uc?id=14-pVDocsabI6XoFDRe0Jdig1VnHDWkkd" width="500"/>


## 붓꽃 데이터셋의 구성

In [ ]:
import pandas as pd

# 붓꽃 데이터 불러오기
df = pd.read_csv('https://archive.ics.uci.edu/ml/'
        'machine-learning-databases/iris/iris.data', header=None)
df.tail()   # 데이터의 끝부분만 출력 


* 하나의 샘플 데이터는 한 줄, 다섯 컬럼으로 구성됨
  * 컬럼 0 : 꽃받침 길이 (sepal length in cm)
  * 컬럼 1 : 꽃받침 폭 (sepal width in cm)
  * 컬럼 2 : 꽃잎 길이 (petal length in cm)
  * 컬럼 3 : 꽃잎 폭 (petal width in cm)
  * 컬럼 4 : 꽃 종류, 즉 정답값에 해당 (Iris Setosa, Iris Versicolour, Iris Virginica)
* 총 150 샘플로 구성
  * 0 ~ 49번째 : Setosa
  * 50 ~ 99번째 : Versicolour
  * 100 ~ 149번째 : Virginica
* **문제** : 길이, 폭 데이터를 이용해 어떤 종류인지 맞출수 있을까?

### 붓꽃 데이터 그래프 그리기

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 꽃받침 길이와 꽃잎 길이를 추출합니다
X = df.iloc[0:100, [0, 2]].values         # 앞 100개 샘플의 컬럼0(꽃받침 길이)과 컬럼2(꽃잎 길이) 값을 가지고 옴

# 산점도를 그립니다. x축: 컬럼0(꽃받침 길이), y축: 컬럼1(꽃잎 길이)
plt.scatter(X[:50, 0], X[:50, 1],
            color='red', marker='o', label='setosa')      # Setosa에 해당하는 샘플(0~49번째)만 플롯팅
plt.scatter(X[50:100, 0], X[50:100, 1],
            color='blue', marker='x', label='versicolor') # Versicolor에 해당하는 샘플(50~99번째)만 플롯팅

# 그래프 텍스트 설정
plt.xlabel('Sepal length [cm]')
plt.ylabel('Petal length [cm]')
plt.legend(loc='upper left')

plt.show()

## 객체 지향 퍼셉트론 API

In [ ]:
class Perceptron(object):
    
    def __init__(self, lr=0.01, n_iter=50, random_state=1):
        self.lr = lr                      # Learning rate (학습률, 0.0~1.0)
        self.n_iter = n_iter              # 훈련 데이터셋 반복 횟수
        self.random_state = random_state  # 난수발생기 시드

    # 입력값에 대해 정답 추론
    def predict(self, X):
        output = np.dot(X, self.w_[1:]) + self.w_[0]
        return np.where(output >= 0.0, 1, -1)    # 출력값이 0 이상이면 1, 아니면 -1을 리턴함        

    def fit(self, X, y):
        # X : 학습 데이터
        # y : 정답값(레이블)

        rgen = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal(loc=0.0, scale=0.01, size=1 + X.shape[1])   # 가중치 랜덤 초기화
        self.errors_ = []   # 에폭마다 누적된 분류 오류

        for _ in range(self.n_iter):      # 지정한 회수(n_iter)만큼 반복
            errors = 0
            for xi, target in zip(X, y):  # 각 샘플에 대해 반복
                update = self.lr * (target - self.predict(xi))
                self.w_[1:] += update * xi    # 가중치 업데이트
                self.w_[0] += update          # 바이어스 업데이트
                errors += int(update != 0.0)  # 에러 계산
            self.errors_.append(errors)
        return self


### 퍼셉트론 모델 훈련하기

In [ ]:
# 퍼셉트론 객체 생성
ppn = Perceptron(lr=0.1, n_iter=10)

# setosa와 versicolor를 선택
y = df.iloc[0:100, 4].values              # 앞 100개 샘플의 정답값(꽃종류) 을 가지고 옴
y = np.where(y == 'Iris-setosa', -1, 1)   # 값이 Iris-setosa이면 -1, 아니면 1로 표기함

# 학습 실행
# X : 앞 100개 샘플의 컬럼0(꽃받침 길이)과 컬럼2(꽃잎 길이) 값. 앞 셀에서 "X = df.iloc[0:100, [0, 2]].values"로 생성함. 
# y : 앞 100개 샘플의 정답값(꽃종류). -1 이면 setosa, 1 이면 veriscolor.
ppn.fit(X, y)

plt.plot(range(1, len(ppn.errors_) + 1), ppn.errors_, marker='o')
plt.xlabel('Epochs')
plt.ylabel('Number of errors')

plt.show()

### 결정 경계 그래프 함수

In [ ]:
from matplotlib.colors import ListedColormap

def plot_decision_regions(X, y, classifier, resolution=0.02):

    # 마커와 컬러맵 설정
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # 결정 경계 그림
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)  # 그리드별 예측결과 산출
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.3, cmap=cmap)                 # 평면에 예측결과 표시
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    # 스캐터플랏 그림
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], 
                    y=X[y == cl, 1],
                    alpha=0.8, 
                    c=colors[idx],
                    marker=markers[idx], 
                    label=cl, 
                    edgecolor='black')

In [ ]:
plot_decision_regions(X, y, classifier=ppn)
plt.xlabel('sepal length [cm]')
plt.ylabel('petal length [cm]')
plt.title('Perceptron: classification - setosa vs. versicolor')
plt.legend(loc='upper left')

plt.show()